## Real-time Object Detection

- Developed a real-time object detection application using Tkinter for GUI and OpenCV for video capture.
- Implemented MobileNet SSD model for accurate object detection, processing frames from webcam in real- time. 
- Designed user interface with buttons for starting/stopping video and object detection processes.
- Integrated Tkinter for GUI and PIL for image processing, displaying real-time object detection results. 
- Utilized OpenCV’s DNN module to process video frames, apply object detection, and draw bounding boxes. 
- Ensured smooth operation by handling video capture, frame processing, and user interactions seamlessly. 
- Technologies Used : Python, OpenCV, Tkinter, MobileNet SSD etc.

In [2]:
import tkinter as tk
from tkinter import messagebox, Label, Entry, Button, Canvas, Toplevel
import cv2
import os
import numpy as np
from PIL import Image, ImageTk
import imutils
import settings


def after_login():
    def start_video():
        if not cap.isOpened():
            messagebox.showerror("Error", "Unable to open the camera.")
            return

        settings.start_video = True
        show_frame()

    def stop_video():
        settings.start_video = False
        settings.start_processing = False
        lmain.config(image='')

    def start_process():
        if not settings.start_video:
            messagebox.showwarning("Warning", "Start video before object detection.")
        else:
            settings.start_processing = True

    def stop_process():
        settings.start_processing = False

    def show_frame():
        if not settings.start_video:
            return

        ret, frame = cap.read()
        if not ret:
            messagebox.showerror("Error", "Failed to grab frame.")
            return

        frame = cv2.flip(frame, 1)
        frame = imutils.resize(frame, width=400)

        if settings.start_processing:
            frame = process_frame(frame)

        cv2image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(cv2image)
        imgtk = ImageTk.PhotoImage(image=img)
        lmain.imgtk = imgtk
        lmain.configure(image=imgtk)
        lmain.after(10, show_frame)

    def process_frame(img):
        (h, w) = img.shape[:2]
        blob = cv2.dnn.blobFromImage(cv2.resize(img, (300, 300)), 0.007843, (300, 300), 127.5)

        net.setInput(blob)
        detections = net.forward()

        for i in np.arange(0, detections.shape[2]):
            confidence = detections[0, 0, i, 2]

            if confidence > 0.2:
                idx = int(detections[0, 0, i, 1])
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")

                label = f"{CLASSES[idx]}: {confidence * 100:.2f}%"
                cv2.rectangle(img, (startX, startY), (endX, endY), COLORS[idx], 2)
                y = startY - 15 if startY - 15 > 15 else startY + 15
                cv2.putText(img, label, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLORS[idx], 2)

        return img

    CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat", "bottle", "bus", "car", "cat", "chair", "cow",
               "diningtable", "dog", "horse", "motorbike", "person", "pottedplant", "sheep", "sofa", "train", "tvmonitor"]
    COLORS = np.random.uniform(0, 255, size=(len(CLASSES), 3))

    try:
        print("Loading model...")
        net = cv2.dnn.readNetFromCaffe('MobileNetSSD_deploy.prototxt.txt', 'MobileNetSSD_deploy.caffemodel')
    except cv2.error as e:
        messagebox.showerror("Error", "Failed to load object detection model.")
        return

    try:
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            raise Exception("Unable to access the camera.")
    except Exception as e:
        messagebox.showerror("Error", str(e))
        return

    # GUI for Object Detection
    window = Toplevel()
    window.title("Object Detection")
    window.geometry('1366x786')

    image = Image.open('bg.jpeg')
    photo_image = ImageTk.PhotoImage(image)
    label = Label(window, image=photo_image)
    label.pack()

    canvas1 = Canvas(window, width=400, height=300, relief='raised')
    canvas1.pack()

    lbl = tk.Label(window, text="Real Time Object Detection", font=('helvetica', 21, 'bold'),
                   height=2, width=60, bg='deep sky blue', fg='white')
    lbl.place(x=660, y=100, anchor="center")

    imageFrame = tk.Frame(window, width=600, height=500)
    imageFrame.place(x=410, y=150)

    lmain = tk.Label(imageFrame, text="Press Start Video..")
    lmain.grid(row=1, column=1)

    startVideoStreamBtn = tk.Button(window, text="Start Video", height=1, width=10, command=start_video, bg='white',
                                    fg='black', font=('helvetica', 15, 'bold'))
    startVideoStreamBtn.place(x=300, y=500)

    stopVideoStreamBtn = tk.Button(window, text="Stop Video", height=1, width=10, command=stop_video, bg='white',
                                   fg='black', font=('helvetica', 15, 'bold'))
    stopVideoStreamBtn.place(x=300, y=550)

    startProcessBtn = tk.Button(window, text="Start Detection", height=1, width=15, command=start_process, bg='white',
                                fg='black', font=('helvetica', 15, 'bold'))
    startProcessBtn.place(x=800, y=500)

    stopProcessBtn = tk.Button(window, text="Stop Detection", height=1, width=15, command=stop_process, bg='white',
                               fg='black', font=('helvetica', 15, 'bold'))
    stopProcessBtn.place(x=800, y=550)

    def on_closing():
        stop_video()
        cap.release()
        window.destroy()

    window.protocol("WM_DELETE_WINDOW", on_closing)
    window.mainloop()


def login_req():
    if E1.get() == "Admin" and E2.get() == "1234":
        after_login()
    else:
        chkpass = tk.Label(window, text="Username or Password Not valid", font=('helvetica', 19, 'bold'),
                           height=3, width=190, bg='brown', fg='white')
        chkpass.place(x=350, y=210, anchor="center")


def close_window():
    if messagebox.askyesno("Exit Application", "Are you sure you want to exit the application?"):
        window.destroy()


# LOGIN PAGE
window = tk.Tk()
window.title("Object Detection")
window.geometry("1366x786")

image = Image.open('bg.jpeg')
photo_image = ImageTk.PhotoImage(image)
label = Label(window, image=photo_image)
label.pack()

canvas1 = Canvas(window, width=400, height=300, relief='raised')
canvas1.pack()

quitButton = Button(window, text="Quit", height=1, width=10, command=close_window, bg='white',
                    fg='black', font=('helvetica', 15, 'bold'))
quitButton.place(x=550, y=390)

loginButton = Button(window, text="Login", height=1, width=10, command=login_req, bg='white',
                     fg='black', font=('helvetica', 15, 'bold'))
loginButton.place(x=700, y=390)

lbl1 = Label(window, text="Password", height=1, width=10, font=('helvetica', 15, 'bold'), bg='white', fg='black')
lbl1.place(x=550, y=350, anchor="center")

lbl2 = Label(window, text="Username", font=('helvetica', 15, 'bold'), height=1, width=10, bg='white', fg='black')
lbl2.place(x=550, y=280, anchor="center")

lbl3 = Label(window, text="Welcome to Real Time Object Detection\nLogin to proceed", font=('helvetica', 15, 'bold'),
             height=2, width=60, bg='deep sky blue', fg='white')
lbl3.place(x=660, y=540, anchor="center")

lbl4 = Label(window, text="Real Time Object Detection", font=('helvetica', 21, 'bold'),
             height=2, width=60, bg='deep sky blue', fg='white')
lbl4.place(x=660, y=100, anchor="center")

E1 = Entry(window, font=('helvetica', 12, 'bold'))
E1.place(x=740, y=280, anchor="center")

E2 = Entry(window, show="*", font=('helvetica', 12, 'bold'))
E2.place(x=740, y=350, anchor="center")

window.mainloop()


Loading model...
